# Distribution (Generalization performance by TRAIN vs TEST)

In [8]:
import pandas as pd

df = pd.DataFrame(index=[idx for idx in range(20)])

with open("../scripts/RESULTS_230227/RUN_221213_DIST/res.txt") as f:
    ls = f.readlines()
    for line in ls:
        ds = line.split("\t")

        vals = eval(ds[1])

        col_rmse = ds[0]+"_rmse"
        col_r2 = ds[0]+"_r2"

        df.loc[:, col_rmse] = pd.Series([val[0] for val in vals])
        df.loc[:, col_r2] = pd.Series([val[1] for val in vals])


In [9]:
import re
import seaborn as sns
import matplotlib.pyplot as plt

mmss = [f"{idx:02}" for idx in range(1, 16, 1)]
mtxs = ["rmse", "r2"]

SHOW_CHART = False

if SHOW_CHART:
    for mms in mmss:

        print(mms)
        fig, axes = plt.subplots(1, 2, figsize=(16, 6))

        for idx, mtx in enumerate(mtxs):
            col_train = f"{mms}_train_{mtx}"
            col_test = f"{mms}_test_{mtx}"

            if col_train not in df.columns or col_test not in df.columns:
                continue
            
            x1 = df[f"{mms}_train_{mtx}"]
            x2 = df[f"{mms}_test_{mtx}"]

            axes[idx].hist([x1, x2], bins=10, label=[f"MMS{mms}_FV_{mtx}_train", f"MMS{mms}_FV_{mtx}_test"])

            axes[idx].legend()

        plt.show()
    # df[df.columns.str.match("^01_test_r2")[0]].hist(alpha=0.5)

# print(df.mean())

"""setup result table"""
se_mean = df.mean()
se_std = df.std()
df_mean = pd.DataFrame()
df_std = pd.DataFrame()
rname = "XX"
for idx in se_mean.index:
    rname = idx[0:2]
    cname = re.sub("^.._", "", idx)
    # print(rname, cname)
    df_mean.loc[rname, cname] = se_mean[idx]
    df_std.loc[rname, cname+"_std"] = se_std[idx]

df_mean = df_mean.dropna()
df_std = df_std.dropna()

df_rmse = pd.concat([df_mean.loc[:, ["train_rmse"]], df_std.loc[:, ["train_rmse_std"]], df_mean.loc[:, ["test_rmse"]], df_std.loc[:, ["test_rmse_std"]]], axis=1)
df_r2   = pd.concat([df_mean.loc[:, ["train_r2"]], df_std.loc[:, ["train_r2_std"]], df_mean.loc[:, ["test_r2"]], df_std.loc[:, ["test_r2_std"]]], axis=1)

print(df_rmse.dtypes)

cm_g  = sns.light_palette('green', as_cmap=True)
cm_gr = sns.light_palette('blue', reverse=True, as_cmap=True)
df_rmse = df_rmse.style.background_gradient(cmap=cm_g, axis=0)
df_r2 = df_r2.style.background_gradient(cmap=cm_gr, axis=0)
# # display(df_mean)
# # display(df_std)
# df_mean
display(df_rmse)
display(df_r2)

train_rmse        float64
train_rmse_std    float64
test_rmse         float64
test_rmse_std     float64
dtype: object


,train_rmse,train_rmse_std,test_rmse,test_rmse_std
01,0.593655,0.035429,0.761824,0.097668
02,0.493978,0.026121,0.722983,0.070135
03,0.597458,0.037650,0.727732,0.071643
05,0.454764,0.021313,0.754612,0.053069
06,0.585087,0.059629,0.639833,0.112711
08,0.673391,0.042827,0.693626,0.095264
09,0.625421,0.022198,1.109286,0.040700
10,0.611254,0.032730,0.940230,0.053316
11,0.699642,0.018915,1.052430,0.063908
12,0.517499,0.019747,0.567268,0.069052


,train_r2,train_r2_std,test_r2,test_r2_std
01,0.493194,0.060914,-0.201009,0.342047
02,0.565728,0.044997,0.303426,0.133319
03,0.486480,0.064475,-0.089011,0.217867
05,0.676832,0.030607,-0.137141,0.159786
06,0.597276,0.080966,0.388486,0.225397
08,0.457648,0.066257,0.113155,0.265761
09,0.492285,0.036353,0.142007,0.063258
10,0.631277,0.038922,-0.359532,0.155892
11,0.411824,0.031916,-1.179523,0.265022
12,0.370444,0.047827,-0.623316,0.446483


In [10]:
# import pandas as pd
# import matplotlib.pyplot as plt
# from matplotlib import colors

# def background_gradient(s, m, M, cmap='PuBu', low=0, high=0):
#     rng = M - m
#     norm = colors.Normalize(m - (rng * low),
#                             M + (rng * high))
#     normed = norm(s.values)
#     c = [colors.rgb2hex(x) for x in plt.cm.get_cmap(cmap)(normed)]
#     return ['background-color: %s' % color for color in c]

# df_mean.style.apply(background_gradient,
#                     map='PuBu',
#                     m=df.min().min(),
#                     M=df.max().max(),
#                     low=0,
#                     high=0.2)

# LIBRARY for MAPPING

In [1]:
def max_bold_min_italic(val):
    is_max = val == val.max()
    is_min = val == val.min()
    ret = ['font-style: italic' if cell else '' for cell in is_min]
    ret = ['font-weight: bold' if cell else ret[idx] for idx, cell in enumerate(is_max)]
    return ret

def min_bold_max_italic(val):
    is_max = val == val.max()
    is_min= val == val.min()
    ret = ['font-weight: bold' if cell else '' for cell in is_min]
    ret = ['font-style: italic' if cell else ret[idx] for idx, cell in enumerate(is_max)]
    return ret

# RESULTS for normal GP(N), FIGP(FV, FVD, FVD2), FIGP(simple SAM for X, C, XC), SVR, MLR
1. Data separation: train:test = 2:8, 5:5, 8:2 (random seed = 0-4)
2. n figures = 4, 7, 10 
3. method nomal GP(N), SVR, MLR, FIGP
4. MMS01-12

In [3]:
import glob
import re
import pandas as pd

# res_files = sorted(glob.glob("../scripts/RESULTS_230307/*/res.txt"))
res_files = sorted(glob.glob("../scripts/RESULTS_230329/*/res.txt"))
# print(res_files)
print("n results:", len(res_files))

PATH_PTN = re.compile(r"^.*/2303.._D([^_]*)_F([^_]*)_(([^_]*)|([^_]*)_([^_]*)|([^_]*)_([^_]*)_([^_]*))/res.txt")
CMET_PTN = re.compile(r"(MLR|SVR)")

res_df = pd.DataFrame(columns=["METHOD", "N FEATURES", "TRAIN RATIO", "PATH"])

for _path in res_files:
    
    if PATH_PTN.match(_path) is not None:
        _match = PATH_PTN.match(_path)
        # print(_match.groups(), _match.group(0),_match.group(1),_match.group(2),_match.group(3))
        _train_r = float(f"0.{_match.group(1)}")
        _n_feats = int(_match.group(2))
        _method = None
        
        if _match.group(4) is not None:
            _method = _match.group(4)
        elif _match.group(5) is not None:
            _lmd = _match.group(6)[1:].split(":")
            _method = f"{_match.group(5)} +{_lmd[0]}*{_match.group(6)[0]}(d={_lmd[1]})"
        elif _match.group(7) is not None:
            _xlmd = _match.group(8)[1:].split(":")
            _clmd = _match.group(9)[1:].split(":")
            _method = f"{_match.group(7)} +{_xlmd[0]}*{_match.group(8)[0]}(d={_xlmd[1]}) +{_clmd[0]}*{_match.group(9)[0]}(d={_clmd[1]})"
        else:
            raise Exception("no match", _match.groups())

        # print(_n_feats)
        res_df.loc[len(res_df)] = {"METHOD":_method, "N FEATURES": _n_feats, "TRAIN RATIO": _train_r, "PATH": _path}
        # print(res_df.loc[len(res_df)])
            
    else:
        raise Exception("no match:", _path)
    

df = pd.DataFrame(columns=["MMS", "METHOD", "N FEATURES", "TRAIN RATIO", "DATASET", "RSEED", "RMSE", "R2"])

for _idx, _vals in res_df.iterrows():
    _method = _vals[0]
    _n_feats = _vals[1]
    _train_r = _vals[2]
    _path = _vals[3]
    # print(_n_feats, _train_r, _method, _path)
        # _df = dfs[_n_feats][_train_r]
    with open(_path) as f:
        ls = f.readlines()
        # print(ls)
        for line in ls:
            ds = line.split("\t")
            
            # print(ds)
            _val = ds[1].replace("inf", "10.0", 10)
            vals = eval(_val)
            # print(_val, vals)

            dss = re.split(r"[/_]", ds[0])
            _mmsno = dss[0]
            _rndseed = int(dss[1])
            _dataset = dss[2]
            # print(dss, _mmsno, _rndseed, _dataset)

            if CMET_PTN.match(_method):
                df.loc[len(df)] = {"MMS": f"MMS{_mmsno}", "METHOD": _method, "N FEATURES": _n_feats, "TRAIN RATIO": _train_r, "DATASET": _dataset, "RSEED": _rndseed, "RMSE": vals[0], "R2": vals[1]}
            else:
                if "train_all" in ds[0]:
                    df.loc[len(df)] = {"MMS": f"MMS{_mmsno}", "METHOD": _method, "N FEATURES": _n_feats, "TRAIN RATIO": _train_r, "DATASET": _dataset, "RSEED": _rndseed, "RMSE": vals[0][1], "R2": vals[0][2]}
                elif "test" in ds[0]:
                    df.loc[len(df)] = {"MMS": f"MMS{_mmsno}", "METHOD": _method, "N FEATURES": _n_feats, "TRAIN RATIO": _train_r, "DATASET": _dataset, "RSEED": _rndseed, "RMSE": vals[0], "R2": vals[1]}
                else:
                    pass

pd.options.display.float_format = "{:.2e}".format
df

n results: 198


,MMS,METHOD,N FEATURES,TRAIN RATIO,DATASET,RSEED,RMSE,R2
0,MMS01,FV,10,2.00e-01,train,0,1.99e-01,9.68e-01
1,MMS01,FV,10,2.00e-01,test,0,5.93e+00,-6.65e+01
2,MMS01,FV,10,2.00e-01,train,1,1.02e-01,9.84e-01
3,MMS01,FV,10,2.00e-01,test,1,2.11e+01,-7.34e+02
4,MMS01,FV,10,2.00e-01,train,2,7.79e-02,9.91e-01
...,...,...,...,...,...,...,...,...
22782,MMS12,MEAN,7,8.00e-01,test,0,6.63e-01,-9.84e-02
22783,MMS12,MEAN,7,8.00e-01,test,1,1.11e+00,-7.98e-02
22784,MMS12,MEAN,7,8.00e-01,test,2,1.07e+00,-1.11e-01
22785,MMS12,MEAN,7,8.00e-01,test,3,8.62e-01,-6.08e-02


In [4]:
df_res = pd.DataFrame(columns=["MMS", "METHOD", "N FEATURES", "TRAIN RATIO", \
                               "RMSE TRAIN COUNT", "RMSE TRAIN MEAN", "RMSE TRAIN STD", "RMSE TRAIN MIN", "RMSE TRAIN 25%", "RMSE TRAIN 50%", "RMSE TRAIN 75%", "RMSE TRAIN MAX", \
                               "R2 TRAIN COUNT",   "R2 TRAIN MEAN",   "R2 TRAIN STD",   "R2 TRAIN MIN",   "R2 TRAIN 25%",   "R2 TRAIN 50%",   "R2 TRAIN 75%",   "R2 TRAIN MAX", \
                               "RMSE TEST COUNT",  "RMSE TEST MEAN",  "RMSE TEST STD",  "RMSE TEST MIN",  "RMSE TEST 25%",  "RMSE TEST 50%",  "RMSE TEST 75%",  "RMSE TEST MAX", \
                               "R2 TEST COUNT",    "R2 TEST MEAN",    "R2 TEST STD",    "R2 TEST MIN",    "R2 TEST 25%",    "R2 TEST 50%",    "R2 TEST 75%",    "R2 TEST MAX"])                                                                                                                                                                            

mmss = df["MMS"].drop_duplicates().values
methods = df["METHOD"].drop_duplicates().values
nfeatss = df["N FEATURES"].drop_duplicates().values
trainrs = df["TRAIN RATIO"].drop_duplicates().values

print(mmss)
print(methods)
print(nfeatss)
print(trainrs)

for _mms in mmss:
    for _method in methods:
        for _nfeats in nfeatss:
            for _trainr in trainrs:
                _train = df[(df["MMS"] == _mms) & (df["METHOD"] == _method) & (df["N FEATURES"] == _nfeats) & (df["TRAIN RATIO"] == _trainr) & (df["DATASET"] == "train")].loc[:, ["RMSE","R2"]].describe()
                _test = df[(df["MMS"] == _mms) & (df["METHOD"] == _method) & (df["N FEATURES"] == _nfeats) & (df["TRAIN RATIO"] == _trainr) & (df["DATASET"] == "test")].loc[:, ["RMSE","R2"]].describe()
                new_row = [_mms, _method, _nfeats, _trainr]+list(_train["RMSE"].values)+list(_train["R2"])+list(_test["RMSE"].values)+list(_test["R2"])
                # print(len(df_res.columns), len(new_row))
                df_res.loc[len(df_res)] = new_row
                
df_res


['MMS01' 'MMS02' 'MMS03' 'MMS04' 'MMS05' 'MMS06' 'MMS07' 'MMS08' 'MMS09'
 'MMS10' 'MMS11' 'MMS12']
['FV' 'FVD' 'FVD2' 'FVD2 +.5*X(d=.1) +.5*C(d=.1)' 'FV +1*C(d=.01)'
 'FV +1*C(d=.02)' 'FV +1*C(d=.05)' 'FV +1*C(d=.1)' 'FV +1*C(d=.2)'
 'FV +1*C(d=.5)' 'FV +.5*X(d=.1) +.5*C(d=.1)' 'FV +1*X(d=.01)'
 'FV +1*X(d=.02)' 'FV +1*X(d=.05)' 'FV +1*X(d=.1)' 'FV +1*X(d=.2)'
 'FV +1*X(d=.5)' 'MLR' 'N' 'N +.5*X(d=.1) +.5*C(d=.1)' 'SVR' 'MEAN']
[10  4  7]
[0.2 0.5 0.8]


,MMS,METHOD,N FEATURES,TRAIN RATIO,RMSE TRAIN COUNT,RMSE TRAIN MEAN,RMSE TRAIN STD,RMSE TRAIN MIN,RMSE TRAIN 25%,RMSE TRAIN 50%,...,RMSE TEST 75%,RMSE TEST MAX,R2 TEST COUNT,R2 TEST MEAN,R2 TEST STD,R2 TEST MIN,R2 TEST 25%,R2 TEST 50%,R2 TEST 75%,R2 TEST MAX
0,MMS01,FV,10,2.00e-01,5.00e+00,1.28e-01,5.34e-02,7.79e-02,9.16e-02,1.02e-01,...,5.93e+00,2.11e+01,5.00e+00,-1.62e+02,3.21e+02,-7.34e+02,-6.65e+01,-6.91e+00,-3.36e+00,-1.40e+00
1,MMS01,FV,10,5.00e-01,5.00e+00,3.97e-01,7.63e-02,3.25e-01,3.47e-01,3.67e-01,...,1.20e+00,1.36e+00,4.00e+00,-1.65e+00,1.21e+00,-3.07e+00,-2.28e+00,-1.67e+00,-1.04e+00,-1.95e-01
2,MMS01,FV,10,8.00e-01,5.00e+00,5.32e-01,6.23e-02,4.63e-01,4.96e-01,5.05e-01,...,9.38e-01,1.40e+00,5.00e+00,-9.01e-02,7.36e-01,-8.98e-01,-7.93e-01,1.21e-02,5.22e-01,7.06e-01
3,MMS01,FV,4,2.00e-01,5.00e+00,1.94e-01,5.82e-02,1.04e-01,1.84e-01,1.93e-01,...,1.11e+06,4.43e+06,4.00e+00,-8.11e+12,1.62e+13,-3.24e+13,-8.11e+12,-2.17e+02,-6.00e+00,-1.15e+00
4,MMS01,FV,4,5.00e-01,5.00e+00,5.22e-01,6.18e-02,4.41e-01,4.99e-01,5.03e-01,...,1.00e+00,3.19e+02,5.00e+00,-4.49e+04,1.00e+05,-2.24e+05,-1.63e+00,-1.11e+00,-4.95e-01,2.77e-01
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2371,MMS12,MEAN,4,5.00e-01,0.00e+00,NaN,NaN,NaN,NaN,NaN,...,1.03e+00,1.10e+00,5.00e+00,-2.13e-01,1.64e-01,-4.39e-01,-2.91e-01,-1.97e-01,-1.31e-01,-5.52e-03
2372,MMS12,MEAN,4,8.00e-01,0.00e+00,NaN,NaN,NaN,NaN,NaN,...,1.07e+00,1.11e+00,5.00e+00,-8.53e-02,1.98e-02,-1.11e-01,-9.84e-02,-7.98e-02,-7.63e-02,-6.08e-02
2373,MMS12,MEAN,7,2.00e-01,0.00e+00,NaN,NaN,NaN,NaN,NaN,...,1.17e+00,1.25e+00,5.00e+00,-5.18e-01,5.00e-01,-1.29e+00,-7.10e-01,-3.35e-01,-2.49e-01,-6.65e-03
2374,MMS12,MEAN,7,5.00e-01,0.00e+00,NaN,NaN,NaN,NaN,NaN,...,1.03e+00,1.10e+00,5.00e+00,-2.13e-01,1.64e-01,-4.39e-01,-2.91e-01,-1.97e-01,-1.31e-01,-5.52e-03


In [5]:
# import matplotlib.pyplot as plt
import seaborn as sns

cm_gr = sns.light_palette('green', reverse=True, as_cmap=True)
cm_br = sns.light_palette('blue', reverse=True, as_cmap=True)
cm_b  = sns.light_palette('blue', reverse=False, as_cmap=True)


for _nfeats in nfeatss:
    for _trainr in trainrs:
        print(f"======= RMSE TEST (N FEATS: {_nfeats}, TRAIN RATIO: {_trainr}) =======")
        df_rmse_test = pd.DataFrame(index=mmss, columns=methods)
        df_rmse_test_med = pd.DataFrame(index=mmss, columns=methods)
        df_rmse_test_count = pd.DataFrame(index=mmss, columns=methods)
        _df = df_res[(df_res["N FEATURES"] == _nfeats) & (df_res["TRAIN RATIO"] == _trainr)]
        for _mms in mmss:
            # display(_df[(_df["MMS"] == _mms)].loc[:, ["METHOD", "RMSE TEST MEAN"]])
            df_rmse_test.loc[_mms] = _df[(_df["MMS"] == _mms)]["RMSE TEST MEAN"].values
            df_rmse_test_med.loc[_mms] = _df[(_df["MMS"] == _mms)]["RMSE TEST 50%"].values
            df_rmse_test_count.loc[_mms] = _df[(_df["MMS"] == _mms)]["RMSE TEST COUNT"].values

        # df_rmse_test_F10_D8_s = df_rmse_test_F10_D8.style.background_gradient(cmap=cm_g, axis=1)
        # s_df_rmse_test = df_rmse_test.astype("float64").style.background_gradient(cmap=cm_gr, axis=1).set_precision(3)
        # s_df_rmse_test_med = df_rmse_test_med.astype("float64").style.background_gradient(cmap=cm_br, axis=1).set_precision(3)
        s_df_rmse_test = df_rmse_test.astype("float64").style.background_gradient(cmap=cm_gr, axis=1)
        s_df_rmse_test_med = df_rmse_test_med.astype("float64").style.background_gradient(cmap=cm_br, axis=1)
        s_df_rmse_test_count = df_rmse_test_count.astype("int").style
        s_df_rmse_test.format("{:.2e}")
        s_df_rmse_test_med.format("{:.2e}")

        # s_train_r2 = df_train_r2.style.background_gradient(cmap=cm_gr, vmax=1.0, axis=1).set_precision(3)
        # s_test_r2 = df_test_r2.style.background_gradient(cmap=cm_gr, vmax=1.0, axis=1).set_precision(3)


        s_df_rmse_test = s_df_rmse_test.apply(min_bold_max_italic, axis=1)
        s_df_rmse_test = s_df_rmse_test.apply(min_bold_max_italic, axis=1)
        s_df_rmse_test_med = s_df_rmse_test_med.apply(min_bold_max_italic, axis=1)
        s_df_rmse_test_med = s_df_rmse_test_med.apply(min_bold_max_italic, axis=1)
        s_df_rmse_test_count = s_df_rmse_test_count.apply(max_bold_min_italic, axis=1)
        s_df_rmse_test_count = s_df_rmse_test_count.apply(max_bold_min_italic, axis=1)
        display(s_df_rmse_test)
        display(s_df_rmse_test_med)
        display(s_df_rmse_test_count)


======= RMSE TEST (N FEATS: 10, TRAIN RATIO: 0.2) =======


,FV,FVD,FVD2,FVD2 +.5*X(d=.1) +.5*C(d=.1),FV +1*C(d=.01),FV +1*C(d=.02),FV +1*C(d=.05),FV +1*C(d=.1),FV +1*C(d=.2),FV +1*C(d=.5),FV +.5*X(d=.1) +.5*C(d=.1),FV +1*X(d=.01),FV +1*X(d=.02),FV +1*X(d=.05),FV +1*X(d=.1),FV +1*X(d=.2),FV +1*X(d=.5),MLR,N,N +.5*X(d=.1) +.5*C(d=.1),SVR,MEAN
MMS01,6.41e+00,1.17e+00,1.69e+00,9.30e-01,2.63e+00,3.19e+00,2.67e+00,5.12e+00,1.39e+00,8.31e+00,1.58e+00,2.00e+00,1.97e+00,1.71e+00,2.50e+00,1.81e+00,1.36e+00,1.76e+05,1.97e+01,3.43e+00,8.81e-01,8.16e-01
MMS02,1.01e+00,8.20e-01,8.60e-01,7.12e-01,8.38e-01,9.46e-01,2.35e+01,1.20e+00,2.47e+00,6.65e-01,2.10e+00,1.84e+00,1.13e+00,2.06e+00,1.01e+00,9.21e-01,9.80e-01,9.03e+00,1.06e+01,7.67e-01,6.89e-01,8.01e-01
MMS03,1.76e+00,8.79e-01,7.08e-01,6.98e-01,1.07e+00,1.70e+00,3.13e+00,1.96e+00,1.24e+00,9.10e-01,1.80e+00,1.80e+00,1.59e+00,1.49e+00,1.05e+00,1.75e+00,8.97e-01,8.38e+00,2.25e+00,2.48e+00,6.13e-01,6.82e-01
MMS04,3.66e+01,9.18e-01,8.36e-01,8.40e-01,1.57e+00,3.50e+00,1.97e+00,9.44e+03,4.52e+00,1.51e+00,1.98e+00,2.76e+00,1.41e+00,1.26e+00,1.87e+00,8.77e-01,8.24e-01,7.52e+00,3.64e+00,4.14e+00,8.10e-01,8.39e-01
MMS05,1.82e+00,1.35e+00,2.24e+00,1.31e+00,4.14e+00,4.17e+01,4.10e+00,2.14e+00,1.70e+00,2.25e+00,2.46e+00,2.90e+00,4.38e+00,2.06e+00,2.84e+01,2.76e+00,1.85e+00,6.21e+00,7.54e+55,1.40e+01,1.02e+00,1.10e+00
MMS06,7.62e+00,1.13e+00,1.01e+00,1.02e+00,9.44e+00,1.35e+00,1.20e+00,3.50e+07,1.20e+00,1.20e+00,1.19e+00,1.62e+01,7.03e+00,5.41e+00,1.20e+00,1.22e+00,1.07e+00,1.61e+00,8.26e+01,4.45e+00,9.10e-01,1.04e+00
MMS07,2.68e+00,1.05e+00,3.25e+21,9.47e-01,1.51e+00,9.24e+00,5.41e+00,2.47e+00,2.21e+00,1.51e+01,9.72e-01,7.50e+05,1.39e+01,1.43e+00,1.41e+00,1.51e+00,1.14e+00,3.17e+00,2.68e+00,1.91e+00,9.63e-01,9.83e-01
MMS08,1.41e+00,1.18e+00,1.46e+00,1.11e+00,1.43e+00,4.00e+00,1.40e+00,2.22e+00,1.25e+00,1.20e+00,1.37e+00,1.55e+00,1.57e+00,1.91e+00,1.29e+00,1.27e+00,1.09e+00,4.73e+00,1.17e+01,3.63e+00,9.47e-01,9.18e-01
MMS09,3.12e+00,9.27e-01,7.83e-01,6.74e-01,1.27e+00,1.22e+00,7.61e-01,1.93e+00,8.90e-01,1.59e+01,3.09e+00,1.63e+00,1.53e+00,2.31e+00,1.39e+00,1.47e+00,6.99e-01,2.01e+01,3.95e+00,1.05e+00,8.24e-01,7.40e-01
MMS10,1.87e+00,9.28e-01,6.72e-01,6.65e-01,8.49e-01,9.41e-01,7.22e-01,1.41e+00,9.83e-01,6.88e+00,6.93e-01,1.20e+00,1.43e+00,8.67e-01,1.19e+00,1.08e+00,9.35e-01,1.16e+00,4.93e+00,8.84e-01,9.94e-01,1.00e+00


,FV,FVD,FVD2,FVD2 +.5*X(d=.1) +.5*C(d=.1),FV +1*C(d=.01),FV +1*C(d=.02),FV +1*C(d=.05),FV +1*C(d=.1),FV +1*C(d=.2),FV +1*C(d=.5),FV +.5*X(d=.1) +.5*C(d=.1),FV +1*X(d=.01),FV +1*X(d=.02),FV +1*X(d=.05),FV +1*X(d=.1),FV +1*X(d=.2),FV +1*X(d=.5),MLR,N,N +.5*X(d=.1) +.5*C(d=.1),SVR,MEAN
MMS01,2.08e+00,1.26e+00,1.18e+00,8.80e-01,1.46e+00,1.93e+00,2.74e+00,2.20e+00,1.43e+00,1.08e+00,1.60e+00,1.72e+00,1.44e+00,1.72e+00,1.88e+00,1.54e+00,1.37e+00,1.93e+01,4.63e+00,3.23e+00,8.45e-01,7.82e-01
MMS02,1.02e+00,8.30e-01,9.00e-01,7.30e-01,7.41e-01,8.67e-01,1.16e+00,1.16e+00,1.13e+00,6.54e-01,1.49e+00,1.02e+00,1.11e+00,9.29e-01,8.65e-01,8.96e-01,8.75e-01,6.87e+00,5.43e+00,7.15e-01,6.95e-01,7.65e-01
MMS03,2.02e+00,8.71e-01,6.37e-01,6.29e-01,1.07e+00,1.52e+00,1.16e+00,1.20e+00,1.08e+00,7.66e-01,1.82e+00,1.23e+00,1.61e+00,1.41e+00,1.04e+00,8.88e-01,9.03e-01,3.08e+00,1.63e+00,1.24e+00,6.29e-01,6.50e-01
MMS04,1.95e+00,9.21e-01,7.84e-01,8.13e-01,9.21e-01,1.89e+00,1.71e+00,9.92e-01,1.02e+00,9.82e-01,8.86e-01,2.61e+00,1.12e+00,1.10e+00,8.74e-01,8.57e-01,7.81e-01,2.82e+00,3.73e+00,2.27e+00,7.88e-01,7.93e-01
MMS05,1.51e+00,1.28e+00,1.91e+00,1.02e+00,1.52e+00,1.58e+00,4.32e+00,1.63e+00,1.41e+00,1.43e+00,2.73e+00,1.96e+00,2.28e+00,2.06e+00,1.18e+01,2.26e+00,1.66e+00,2.79e+00,2.22e+00,6.40e+00,1.00e+00,1.03e+00
MMS06,2.53e+00,1.11e+00,1.01e+00,1.02e+00,2.58e+00,1.28e+00,1.14e+00,2.86e+00,1.19e+00,1.20e+00,1.19e+00,1.80e+01,4.03e+00,1.12e+00,1.10e+00,1.22e+00,1.06e+00,1.51e+00,5.74e+00,1.54e+00,9.14e-01,1.03e+00
MMS07,2.83e+00,1.05e+00,1.92e+00,9.25e-01,1.61e+00,4.68e+00,1.49e+00,2.27e+00,2.02e+00,1.52e+01,1.01e+00,1.85e+00,1.35e+00,1.33e+00,1.31e+00,1.51e+00,1.21e+00,2.07e+00,2.51e+00,1.63e+00,9.55e-01,9.52e-01
MMS08,1.36e+00,1.20e+00,1.03e+00,1.08e+00,1.21e+00,1.73e+00,1.38e+00,1.84e+00,9.78e-01,1.17e+00,1.22e+00,1.43e+00,1.63e+00,1.41e+00,1.28e+00,1.27e+00,1.02e+00,1.96e+00,4.19e+00,1.16e+00,9.04e-01,8.83e-01
MMS09,3.10e+00,8.73e-01,8.06e-01,6.74e-01,1.49e+00,9.15e-01,8.07e-01,1.81e+00,9.45e-01,9.89e-01,1.34e+00,1.27e+00,1.35e+00,1.29e+00,1.04e+00,1.43e+00,6.55e-01,1.47e+01,2.41e+00,8.01e-01,8.21e-01,7.29e-01
MMS10,8.82e-01,8.43e-01,6.82e-01,5.64e-01,9.00e-01,8.80e-01,7.03e-01,1.22e+00,8.77e-01,7.01e+00,7.02e-01,9.01e-01,1.43e+00,7.57e-01,9.01e-01,9.76e-01,8.65e-01,1.16e+00,2.21e+00,7.46e-01,1.00e+00,9.80e-01


,FV,FVD,FVD2,FVD2 +.5*X(d=.1) +.5*C(d=.1),FV +1*C(d=.01),FV +1*C(d=.02),FV +1*C(d=.05),FV +1*C(d=.1),FV +1*C(d=.2),FV +1*C(d=.5),FV +.5*X(d=.1) +.5*C(d=.1),FV +1*X(d=.01),FV +1*X(d=.02),FV +1*X(d=.05),FV +1*X(d=.1),FV +1*X(d=.2),FV +1*X(d=.5),MLR,N,N +.5*X(d=.1) +.5*C(d=.1),SVR,MEAN
MMS01,5,5,5,5,5,5,4,5,5,5,5,5,5,5,5,5,4,5,5,5,5,5
MMS02,4,5,4,4,3,5,4,5,4,3,4,4,4,3,4,4,5,5,5,4,5,5
MMS03,3,5,3,4,2,4,4,4,3,3,5,5,5,5,4,5,5,5,5,4,5,5
MMS04,4,5,5,5,5,4,5,5,5,5,4,5,5,5,5,5,5,5,5,4,5,5
MMS05,5,5,4,3,4,5,5,4,5,3,3,5,4,5,5,5,5,5,5,4,5,5
MMS06,4,5,2,5,4,4,3,3,4,4,4,3,4,5,5,4,5,5,4,3,5,5
MMS07,4,5,3,4,5,4,4,4,4,4,3,4,4,5,5,4,5,5,4,4,5,5
MMS08,4,5,5,5,5,5,5,4,4,5,5,5,4,5,4,4,5,5,5,5,5,5
MMS09,5,5,5,5,5,5,5,4,5,5,5,5,5,5,5,5,5,5,5,5,5,5
MMS10,5,5,4,5,4,4,4,4,3,4,4,5,5,4,4,5,5,5,5,4,5,5


======= RMSE TEST (N FEATS: 10, TRAIN RATIO: 0.5) =======


...

,FV,FVD,FVD2,FVD2 +.5*X(d=.1) +.5*C(d=.1),FV +1*C(d=.01),FV +1*C(d=.02),FV +1*C(d=.05),FV +1*C(d=.1),FV +1*C(d=.2),FV +1*C(d=.5),FV +.5*X(d=.1) +.5*C(d=.1),FV +1*X(d=.01),FV +1*X(d=.02),FV +1*X(d=.05),FV +1*X(d=.1),FV +1*X(d=.2),FV +1*X(d=.5),MLR,N,N +.5*X(d=.1) +.5*C(d=.1),SVR,MEAN
MMS01,4,5,4,5,5,5,5,5,5,4,5,5,5,5,5,4,5,5,5,5,5,5
MMS02,5,5,5,5,5,5,5,5,5,4,5,5,5,5,5,5,5,5,5,5,5,5
MMS03,5,5,4,5,4,5,4,4,4,4,5,4,5,4,5,5,5,5,4,5,5,5
MMS04,5,5,5,5,5,5,5,4,5,5,5,5,5,5,5,5,5,5,5,5,5,5
MMS05,5,5,5,5,5,5,5,5,5,5,5,5,5,5,5,5,5,5,5,5,5,5
MMS06,5,5,5,5,4,5,5,5,5,4,5,5,4,4,5,4,5,5,4,5,5,5
MMS07,5,5,5,5,4,4,5,5,5,5,4,5,5,5,5,5,5,5,5,5,5,5
MMS08,5,5,5,5,4,5,5,4,5,5,5,5,5,5,5,5,5,5,5,5,5,5
MMS09,5,5,5,5,5,5,5,5,5,5,5,5,5,5,5,5,5,5,5,5,5,5
MMS10,5,5,5,5,5,5,5,5,5,5,5,5,5,5,5,5,5,5,5,4,5,5


======= RMSE TEST (N FEATS: 7, TRAIN RATIO: 0.8) =======


,FV,FVD,FVD2,FVD2 +.5*X(d=.1) +.5*C(d=.1),FV +1*C(d=.01),FV +1*C(d=.02),FV +1*C(d=.05),FV +1*C(d=.1),FV +1*C(d=.2),FV +1*C(d=.5),FV +.5*X(d=.1) +.5*C(d=.1),FV +1*X(d=.01),FV +1*X(d=.02),FV +1*X(d=.05),FV +1*X(d=.1),FV +1*X(d=.2),FV +1*X(d=.5),MLR,N,N +.5*X(d=.1) +.5*C(d=.1),SVR,MEAN
MMS01,6.95e-01,7.74e-01,7.53e-01,7.71e-01,7.06e-01,7.02e-01,6.93e-01,8.52e-01,7.81e-01,7.55e-01,7.48e-01,8.14e-01,7.07e-01,7.97e-01,6.91e-01,8.42e-01,8.46e-01,9.66e-01,1.05e+00,9.31e-01,8.81e-01,8.66e-01
MMS02,6.17e-01,5.97e-01,6.13e-01,6.04e-01,6.02e-01,5.41e-01,5.93e-01,5.30e-01,5.82e-01,6.12e-01,5.51e-01,5.82e-01,6.38e-01,6.40e-01,5.60e-01,6.68e-01,6.69e-01,5.85e-01,5.41e-01,6.81e-01,6.43e-01,7.17e-01
MMS03,1.19e+00,5.74e-01,5.80e-01,4.75e-01,6.23e-01,5.84e-01,5.76e-01,7.78e-01,5.91e-01,5.73e-01,6.20e-01,5.98e-01,6.97e-01,5.81e-01,7.63e-01,4.99e-01,5.57e-01,5.38e-01,1.02e+00,1.77e+00,5.20e-01,6.47e-01
MMS04,6.57e-01,6.91e-01,6.32e-01,6.68e-01,6.14e-01,6.48e-01,6.44e-01,6.11e-01,6.67e-01,6.87e-01,6.24e-01,6.74e-01,7.07e-01,6.92e-01,6.19e-01,6.77e-01,6.80e-01,5.92e-01,3.43e+00,6.96e-01,7.95e-01,8.06e-01
MMS05,8.00e-01,8.84e-01,7.52e-01,7.12e-01,7.51e-01,8.44e-01,7.29e-01,7.74e-01,8.16e-01,7.78e-01,7.38e-01,8.46e-01,8.33e-01,7.71e-01,7.87e-01,7.57e-01,7.33e-01,8.34e-01,1.24e+00,9.59e-01,8.94e-01,9.27e-01
MMS06,9.29e-01,8.88e-01,9.40e-01,9.11e-01,9.18e-01,1.04e+00,9.68e-01,9.07e-01,1.06e+00,9.00e-01,9.72e-01,9.33e-01,9.17e-01,9.43e-01,1.00e+00,9.59e-01,1.07e+00,9.71e-01,9.43e-01,1.21e+00,9.34e-01,1.00e+00
MMS07,9.28e-01,8.78e-01,9.09e-01,8.23e-01,8.46e-01,8.13e-01,8.19e-01,8.24e-01,8.51e-01,7.94e-01,8.21e-01,8.86e-01,8.13e-01,8.75e-01,8.29e-01,8.35e-01,8.15e-01,8.34e-01,1.23e+00,8.21e-01,8.26e-01,8.90e-01
MMS08,1.06e+00,1.07e+00,1.07e+00,9.73e-01,1.05e+00,1.01e+00,1.04e+00,1.06e+00,1.03e+00,1.06e+00,1.83e+00,1.14e+00,1.04e+00,1.05e+00,9.86e-01,9.99e-01,1.01e+00,1.00e+00,3.19e+00,1.16e+00,9.88e-01,9.94e-01
MMS09,2.05e+00,8.01e-01,6.35e-01,6.44e-01,6.39e-01,6.97e-01,6.42e-01,6.76e-01,7.09e-01,7.12e-01,6.39e-01,1.03e+00,6.24e-01,6.90e-01,6.16e-01,6.40e-01,6.39e-01,6.59e-01,7.29e-01,6.43e-01,6.29e-01,6.07e-01
MMS10,5.99e-01,6.49e-01,5.21e-01,4.97e-01,5.34e-01,5.80e-01,5.42e-01,5.48e-01,6.82e-01,5.55e-01,5.46e-01,5.42e-01,5.39e-01,5.69e-01,5.37e-01,5.72e-01,5.38e-01,5.77e-01,9.90e-01,5.39e-01,7.06e-01,8.46e-01


,FV,FVD,FVD2,FVD2 +.5*X(d=.1) +.5*C(d=.1),FV +1*C(d=.01),FV +1*C(d=.02),FV +1*C(d=.05),FV +1*C(d=.1),FV +1*C(d=.2),FV +1*C(d=.5),FV +.5*X(d=.1) +.5*C(d=.1),FV +1*X(d=.01),FV +1*X(d=.02),FV +1*X(d=.05),FV +1*X(d=.1),FV +1*X(d=.2),FV +1*X(d=.5),MLR,N,N +.5*X(d=.1) +.5*C(d=.1),SVR,MEAN
MMS01,7.43e-01,7.34e-01,8.48e-01,8.30e-01,6.96e-01,7.08e-01,7.65e-01,7.77e-01,7.06e-01,7.50e-01,6.74e-01,7.84e-01,5.33e-01,8.46e-01,7.96e-01,7.82e-01,8.57e-01,8.69e-01,1.11e+00,9.16e-01,8.34e-01,8.48e-01
MMS02,5.60e-01,5.92e-01,5.77e-01,5.78e-01,5.52e-01,4.88e-01,5.34e-01,4.74e-01,5.36e-01,5.62e-01,5.05e-01,5.61e-01,6.53e-01,5.73e-01,5.53e-01,6.45e-01,7.12e-01,5.69e-01,5.49e-01,7.03e-01,6.40e-01,7.21e-01
MMS03,8.64e-01,6.21e-01,5.89e-01,4.52e-01,6.39e-01,6.30e-01,5.23e-01,5.16e-01,5.92e-01,5.39e-01,5.65e-01,6.11e-01,6.19e-01,5.09e-01,6.06e-01,4.54e-01,5.41e-01,5.20e-01,8.77e-01,7.67e-01,5.05e-01,6.46e-01
MMS04,6.30e-01,6.43e-01,6.20e-01,6.84e-01,6.10e-01,6.52e-01,6.12e-01,6.30e-01,6.97e-01,6.63e-01,6.65e-01,7.12e-01,7.62e-01,6.22e-01,6.03e-01,6.67e-01,6.96e-01,6.14e-01,2.69e+00,7.39e-01,8.27e-01,7.45e-01
MMS05,8.32e-01,9.04e-01,7.22e-01,7.24e-01,7.01e-01,8.06e-01,7.64e-01,7.86e-01,8.09e-01,6.40e-01,6.67e-01,7.46e-01,8.46e-01,7.35e-01,7.86e-01,7.50e-01,6.80e-01,9.08e-01,9.45e-01,8.54e-01,8.53e-01,9.29e-01
MMS06,9.82e-01,9.37e-01,8.85e-01,9.53e-01,9.90e-01,1.00e+00,9.55e-01,9.37e-01,9.81e-01,9.11e-01,9.83e-01,1.00e+00,9.45e-01,9.44e-01,9.79e-01,1.00e+00,1.03e+00,9.84e-01,9.97e-01,1.12e+00,9.36e-01,1.01e+00
MMS07,7.81e-01,8.09e-01,8.08e-01,7.74e-01,8.03e-01,7.67e-01,7.95e-01,7.51e-01,8.27e-01,7.51e-01,7.77e-01,8.68e-01,7.58e-01,8.32e-01,7.76e-01,8.34e-01,7.70e-01,7.90e-01,1.29e+00,7.58e-01,7.84e-01,8.40e-01
MMS08,1.07e+00,1.09e+00,1.08e+00,9.80e-01,1.12e+00,9.94e-01,1.04e+00,1.06e+00,1.04e+00,1.03e+00,1.02e+00,1.04e+00,1.06e+00,1.04e+00,1.00e+00,1.01e+00,1.03e+00,1.02e+00,1.18e+00,1.06e+00,9.61e-01,9.84e-01
MMS09,6.68e-01,8.87e-01,6.71e-01,6.86e-01,6.61e-01,7.22e-01,7.04e-01,6.97e-01,7.12e-01,7.42e-01,6.99e-01,7.33e-01,6.65e-01,7.45e-01,6.34e-01,6.84e-01,6.41e-01,7.24e-01,7.77e-01,6.50e-01,6.04e-01,5.89e-01
MMS10,5.68e-01,6.67e-01,5.09e-01,4.31e-01,4.69e-01,5.93e-01,4.93e-01,4.73e-01,6.49e-01,5.71e-01,4.70e-01,4.65e-01,4.99e-01,5.39e-01,4.70e-01,4.97e-01,5.21e-01,5.26e-01,5.95e-01,4.85e-01,6.99e-01,8.47e-01


,FV,FVD,FVD2,FVD2 +.5*X(d=.1) +.5*C(d=.1),FV +1*C(d=.01),FV +1*C(d=.02),FV +1*C(d=.05),FV +1*C(d=.1),FV +1*C(d=.2),FV +1*C(d=.5),FV +.5*X(d=.1) +.5*C(d=.1),FV +1*X(d=.01),FV +1*X(d=.02),FV +1*X(d=.05),FV +1*X(d=.1),FV +1*X(d=.2),FV +1*X(d=.5),MLR,N,N +.5*X(d=.1) +.5*C(d=.1),SVR,MEAN
MMS01,5,5,5,5,5,5,5,5,5,5,5,5,5,5,5,5,5,5,5,5,5,5
MMS02,5,5,5,5,5,5,5,5,5,5,5,5,5,5,5,5,5,5,5,5,5,5
MMS03,5,5,5,5,5,5,5,5,5,5,5,5,5,5,5,5,5,5,5,5,5,5
MMS04,5,5,5,5,5,5,5,5,5,5,5,5,5,5,5,5,5,5,5,5,5,5
MMS05,5,5,5,5,5,5,5,5,5,5,5,5,5,5,5,5,5,5,5,5,5,5
MMS06,5,5,5,5,5,4,4,5,4,5,5,5,5,5,5,5,5,5,5,5,5,5
MMS07,5,5,5,5,5,5,5,5,5,5,5,5,5,5,5,5,5,5,5,5,5,5
MMS08,5,5,5,5,5,5,5,5,5,5,5,5,5,5,5,5,5,5,5,5,5,5
MMS09,5,5,5,5,5,5,5,5,5,5,5,5,5,5,5,5,5,5,5,5,5,5
MMS10,5,5,5,5,5,5,5,5,5,5,5,5,5,5,5,5,5,5,5,5,5,5


# RESULTS (obsolite)

In [121]:
import pandas as pd


res_dict = {
    "SVR": "../scripts/RESULTS_230227/res_svr.txt",
    "MLR": "../scripts/RESULTS_230227/res_mlr.txt",
    "N": "../scripts/RESULTS_230227/RUN_230203_N/res.txt",
    "FV": "../scripts/RESULTS_230227/RUN_230203_FV/res.txt",
    "FV_X(1,.5)": "../scripts/RESULTS_230227/RUN_230224_FV_X_5/res.txt",    
    "FV_X(1,.1)": "../scripts/RESULTS_230227/RUN_230224_FV_X_1/res.txt",
    "FV_X(1,.01)": "../scripts/RESULTS_230227/RUN_230224_FV_X_01/res.txt",
    "FV_X(1,.001)": "../scripts/RESULTS_230227/RUN_230224_FV_X_001/res.txt",
    "FV_X(.1,.1)": "../scripts/RESULTS_230227/RUN_230224_FV_X1_1/res.txt",
    "FV_C(1,.1)": "../scripts/RESULTS_230227/RUN_230224_FV_C_1/res.txt",
    "FV_C(1,.01)": "../scripts/RESULTS_230227/RUN_230224_FV_C_01/res.txt",
    "FV_C(.1,.1)": "../scripts/RESULTS_230227/RUN_230224_FV_C1_1/res.txt",
    "FV_C(.1,.01)": "../scripts/RESULTS_230227/RUN_230224_FV_C1_01/res.txt",
    "FV_X(.5,.1) +C(.5,.1)": "../scripts/RESULTS_230227/RUN_230224_FV_X5_1_C5_1/res.txt",
    "FVD": "../scripts/RESULTS_230227/RUN_230123_FVD/res.txt",
    "FVD2": "../scripts/RESULTS_230227/RUN_230224_FVD2/res.txt",
    "FVD2_X(1,.1)": "../scripts/RESULTS_230227/RUN_230224_FVD2_X_1/res.txt",
    "FVD2_C(1,.1)": "../scripts/RESULTS_230227/RUN_230224_FVD2_C_1/res.txt",
    "FVD2_X(.5,.1) +C(.5,.1)": "../scripts/RESULTS_230227/RUN_230224_FVD2_X5_1_C5_1/res.txt",
    ###
}

df = pd.DataFrame()

for method, resfile in res_dict.items():

    print(method, resfile)
    with open(resfile) as f:
        ls = f.readlines()
        # print(ls)
        for line in ls:
            ds = line.split("\t")

            if "train_all" in ds[0]:
                continue

            # XXXXXX
            _val = ds[1].replace("inf", "10.0", 10)
            vals = eval(_val)
            # vals = eval(ds[1])

            row_rmse = "MMS"+ds[0]+"_rmse"
            row_r2 = "MMS"+ds[0]+"_r2"

            df.loc[row_rmse, method] = vals[0]
            df.loc[row_r2, method] = vals[1]


# df.index

SVR ../scripts/RESULTS_230227/res_svr.txt
MLR ../scripts/RESULTS_230227/res_mlr.txt
N ../scripts/RESULTS_230227/RUN_230203_N/res.txt
FV ../scripts/RESULTS_230227/RUN_230203_FV/res.txt
FV_X(1,.5) ../scripts/RESULTS_230227/RUN_230224_FV_X_5/res.txt
FV_X(1,.1) ../scripts/RESULTS_230227/RUN_230224_FV_X_1/res.txt
FV_X(1,.01) ../scripts/RESULTS_230227/RUN_230224_FV_X_01/res.txt
FV_X(1,.001) ../scripts/RESULTS_230227/RUN_230224_FV_X_001/res.txt
FV_X(.1,.1) ../scripts/RESULTS_230227/RUN_230224_FV_X1_1/res.txt
FV_C(1,.1) ../scripts/RESULTS_230227/RUN_230224_FV_C_1/res.txt
FV_C(1,.01) ../scripts/RESULTS_230227/RUN_230224_FV_C_01/res.txt
FV_C(.1,.1) ../scripts/RESULTS_230227/RUN_230224_FV_C1_1/res.txt
FV_C(.1,.01) ../scripts/RESULTS_230227/RUN_230224_FV_C1_01/res.txt
FV_X(.5,.1) +C(.5,.1) ../scripts/RESULTS_230227/RUN_230224_FV_X5_1_C5_1/res.txt
FVD ../scripts/RESULTS_230227/RUN_230123_FVD/res.txt
FVD2 ../scripts/RESULTS_230227/RUN_230224_FVD2/res.txt
FVD2_X(1,.1) ../scripts/RESULTS_230227/RUN_23

In [122]:
import matplotlib.pyplot as plt
import seaborn as sns

mmss = [f"{idx:02}" for idx in range(1, 16, 1)]
mtxs = ["rmse", "r2"]

"""remove duplication: MMS04, MMS08, MMS10 (MMS14 redo)"""
df = df.filter(regex="^(?!.*(MMS04|MMS08|MMS10)).*$", axis="index")
print(df.index)

DISPLAY_CHART=False
DO_SCATTER=False

if DISPLAY_CHART:
    for mms in mmss:

        print(mms)
        fig, axes = plt.subplots(1, 2, figsize=(16, 6))

        for idx, mtx in enumerate(mtxs):
            row_train = f"MMS{mms}_train_{mtx}"
            row_test = f"MMS{mms}_test_{mtx}"

            if row_train not in df.index or row_test not in df.index:
                continue

            _df = df.loc[[row_train,row_test],:].T

            if DO_SCATTER:
                # x1 = df[f"{mms}_train_{mtx}"]
                # x2 = df[f"{mms}_test_{mtx}"]

                _df.plot.scatter(x=row_train, y=row_test, ax=axes[idx], c=[_j for _j in range(len(list(_df.index)))], alpha=0.8, s=100)

                for _jdx, _label in enumerate(list(_df.index)):
                    axes[idx].text(_df.iloc[_jdx,0], _df.iloc[_jdx,1], _label, alpha=0.75)
                
                _xlim = axes[idx].get_xlim()
                _ylim = axes[idx].get_ylim()
                axes[idx].set_xlim(min(_xlim[0],_ylim[0]), max(_xlim[1],_ylim[1]))
                axes[idx].set_ylim(min(_xlim[0],_ylim[0]), max(_xlim[1],_ylim[1]))
                _xlim = axes[idx].get_xlim()
                _ylim = axes[idx].get_ylim()
                axes[idx].plot(_xlim, _ylim, color="black", lw=.5)
            else:
                _df.T.plot.bar(ax=axes[idx])
                # axes[idx].bar([x1, x2], label=[f"MMS{mms}_FV_{mtx}_train", f"MMS{mms}_FV_{mtx}_test"])


        plt.show()


"""setup result table"""
# display(df)

# pd.options.display.float_format = '{:.1f}'.format

df_train_rmse = df.filter(regex="train_rmse$", axis="index")
df_train_r2 = df.filter(regex="train_r2$", axis="index")
df_test_rmse = df.filter(regex="test_rmse$", axis="index")
df_test_r2 = df.filter(regex="test_r2$", axis="index")

# print(df_train_rmse.dtypes)
# _df = df_train_rmse.astype("float64")
# print(_df.dtypes)

cm_g  = sns.light_palette('green', reverse=True, as_cmap=True)
cm_gr = sns.light_palette('blue', reverse=False, as_cmap=True)
s_train_rmse = df_train_rmse.style.background_gradient(cmap=cm_g, axis=1).set_precision(3)
s_test_rmse = df_test_rmse.style.background_gradient(cmap=cm_g, axis=1).set_precision(3)
s_train_r2 = df_train_r2.style.background_gradient(cmap=cm_gr, vmax=1.0, axis=1).set_precision(3)
s_test_r2 = df_test_r2.style.background_gradient(cmap=cm_gr, vmax=1.0, axis=1).set_precision(3)



# pd.set_option("display.precision", 4)
# pd.options.display.precision = 6
# print(df_train_rmse.dtypes)


s_train_rmse = s_train_rmse.apply(min_bold_max_italic, axis=1)
s_test_rmse = s_test_rmse.apply(min_bold_max_italic, axis=1)
s_train_r2 = s_train_r2.apply(max_bold_min_italic, axis=1)
s_test_r2 = s_test_r2.apply(max_bold_min_italic, axis=1)

display(s_train_rmse)
display(s_test_rmse)
display(s_train_r2)
display(s_test_r2)
# df_train_rmse

Index(['MMS01_train_rmse', 'MMS01_train_r2', 'MMS01_test_rmse',
       'MMS01_test_r2', 'MMS02_train_rmse', 'MMS02_train_r2',
       'MMS02_test_rmse', 'MMS02_test_r2', 'MMS03_train_rmse',
       'MMS03_train_r2', 'MMS03_test_rmse', 'MMS03_test_r2',
       'MMS05_train_rmse', 'MMS05_train_r2', 'MMS05_test_rmse',
       'MMS05_test_r2', 'MMS06_train_rmse', 'MMS06_train_r2',
       'MMS06_test_rmse', 'MMS06_test_r2', 'MMS07_train_rmse',
       'MMS07_train_r2', 'MMS07_test_rmse', 'MMS07_test_r2',
       'MMS09_train_rmse', 'MMS09_train_r2', 'MMS09_test_rmse',
       'MMS09_test_r2', 'MMS11_train_rmse', 'MMS11_train_r2',
       'MMS11_test_rmse', 'MMS11_test_r2', 'MMS12_train_rmse',
       'MMS12_train_r2', 'MMS12_test_rmse', 'MMS12_test_r2',
       'MMS13_train_rmse', 'MMS13_train_r2', 'MMS13_test_rmse',
       'MMS13_test_r2', 'MMS14_train_rmse', 'MMS14_train_r2',
       'MMS14_test_rmse', 'MMS14_test_r2', 'MMS15_train_rmse',
       'MMS15_train_r2', 'MMS15_test_rmse', 'MMS15_test_r2'],

/tmp/ipykernel_25798/423571345.py:69: FutureWarning: this method is deprecated in favour of `Styler.format(precision=..)`
  s_train_rmse = df_train_rmse.style.background_gradient(cmap=cm_g, axis=1).set_precision(3)
/tmp/ipykernel_25798/423571345.py:70: FutureWarning: this method is deprecated in favour of `Styler.format(precision=..)`
  s_test_rmse = df_test_rmse.style.background_gradient(cmap=cm_g, axis=1).set_precision(3)
/tmp/ipykernel_25798/423571345.py:71: FutureWarning: this method is deprecated in favour of `Styler.format(precision=..)`
  s_train_r2 = df_train_r2.style.background_gradient(cmap=cm_gr, vmax=1.0, axis=1).set_precision(3)
/tmp/ipykernel_25798/423571345.py:72: FutureWarning: this method is deprecated in favour of `Styler.format(precision=..)`
  s_test_r2 = df_test_r2.style.background_gradient(cmap=cm_gr, vmax=1.0, axis=1).set_precision(3)


,SVR,MLR,N,FV,"FV_X(1,.5)","FV_X(1,.1)","FV_X(1,.01)","FV_X(1,.001)","FV_X(.1,.1)","FV_C(.1,.1)","FV_C(.1,.01)","FV_X(.5,.1) +C(.5,.1)",FVD,FVD2,"FVD2_X(1,.1)","FVD2_C(1,.1)","FVD2_X(.5,.1) +C(.5,.1)"
MMS01_train_rmse,0.835,0.703,0.463,0.558,0.740,0.578,0.558,0.576,0.563,0.626,0.545,0.669,0.565,0.690,0.728,0.666,0.715
MMS02_train_rmse,0.604,0.521,0.387,0.478,0.669,0.560,0.480,0.450,0.485,0.489,0.426,0.563,0.450,0.483,0.565,0.548,0.543
MMS03_train_rmse,0.835,0.703,0.441,0.566,0.788,0.571,0.580,0.560,0.564,0.633,0.591,0.716,0.580,0.707,0.744,0.668,0.736
MMS05_train_rmse,0.684,0.433,0.398,0.435,0.577,0.467,0.429,0.416,0.426,0.441,0.414,0.479,0.438,0.462,0.531,0.460,0.540
MMS06_train_rmse,0.704,0.561,0.400,0.536,0.753,0.553,0.503,0.462,0.487,0.601,0.512,0.602,0.563,0.622,0.661,0.634,0.681
MMS07_train_rmse,0.817,0.792,0.669,0.768,0.970,0.845,0.751,nan,0.796,0.770,0.746,0.823,0.783,0.796,0.870,0.796,0.834
MMS09_train_rmse,0.776,0.613,0.538,0.594,0.661,0.612,0.600,0.577,0.622,0.634,0.613,0.652,0.637,0.651,0.675,0.678,0.690
MMS11_train_rmse,0.820,0.673,0.639,0.672,0.783,0.726,0.682,0.675,0.684,0.684,0.681,0.752,0.692,0.685,0.753,0.706,0.734
MMS12_train_rmse,0.555,0.532,0.319,0.476,0.570,0.534,0.520,0.499,0.502,0.534,0.497,0.567,0.481,0.532,0.540,0.531,0.534
MMS13_train_rmse,0.649,0.382,0.364,0.367,0.506,0.413,0.393,0.371,0.385,0.419,0.396,0.433,0.356,0.419,0.448,0.471,0.460


,SVR,MLR,N,FV,"FV_X(1,.5)","FV_X(1,.1)","FV_X(1,.01)","FV_X(1,.001)","FV_X(.1,.1)","FV_C(.1,.1)","FV_C(.1,.01)","FV_X(.5,.1) +C(.5,.1)",FVD,FVD2,"FVD2_X(1,.1)","FVD2_C(1,.1)","FVD2_X(.5,.1) +C(.5,.1)"
MMS01_test_rmse,0.804,0.787,1.778,0.742,0.808,0.796,0.867,0.697,0.821,0.752,2.030,0.693,0.795,0.856,0.706,0.770,0.829
MMS02_test_rmse,0.733,0.675,0.812,0.784,0.795,0.747,0.629,0.746,0.763,0.819,0.680,0.665,0.718,0.693,0.666,0.620,0.655
MMS03_test_rmse,0.798,0.787,0.709,0.641,0.839,0.835,0.885,0.739,0.736,0.694,0.732,0.830,0.692,0.835,0.879,0.738,0.808
MMS05_test_rmse,0.827,0.731,0.798,0.775,0.748,0.785,0.815,0.796,0.858,0.753,0.778,0.725,0.748,0.711,0.772,0.878,0.802
MMS06_test_rmse,0.853,0.918,2.404,0.884,0.600,0.955,0.685,0.704,0.819,0.964,0.545,0.701,0.654,0.486,0.619,0.622,0.576
MMS07_test_rmse,0.723,0.762,0.675,0.725,0.822,0.955,0.648,nan,0.848,0.782,0.795,0.799,0.748,0.745,0.706,0.745,0.762
MMS09_test_rmse,1.129,1.071,1.101,1.119,1.047,1.138,1.158,1.366,1.105,1.088,1.101,1.095,1.086,1.076,1.051,1.047,1.059
MMS11_test_rmse,0.764,1.065,10.281,1.086,1.067,1.002,1.174,1.052,1.118,1.156,1.123,1.024,0.953,0.963,0.984,1.046,1.054
MMS12_test_rmse,0.564,0.586,0.788,0.518,0.574,0.562,0.550,0.545,0.537,0.587,0.529,0.588,0.850,0.641,0.571,0.582,0.578
MMS13_test_rmse,0.626,0.507,0.414,0.486,0.413,0.480,0.474,0.477,0.479,0.538,0.440,0.470,0.462,0.481,0.453,0.445,0.466


,SVR,MLR,N,FV,"FV_X(1,.5)","FV_X(1,.1)","FV_X(1,.01)","FV_X(1,.001)","FV_X(.1,.1)","FV_C(.1,.1)","FV_C(.1,.01)","FV_X(.5,.1) +C(.5,.1)",FVD,FVD2,"FVD2_X(1,.1)","FVD2_C(1,.1)","FVD2_X(.5,.1) +C(.5,.1)"
MMS01_train_r2,0.001,0.292,0.693,0.553,0.711,0.540,0.543,0.576,0.550,0.615,0.535,0.617,0.543,0.690,0.630,0.666,0.672
MMS02_train_r2,0.352,0.519,0.734,0.595,0.665,0.538,0.469,0.448,0.477,0.486,0.424,0.526,0.640,0.483,0.495,0.493,0.508
MMS03_train_r2,0.001,0.292,0.721,0.540,0.770,0.521,0.565,0.559,0.560,0.623,0.590,0.679,0.519,0.707,0.696,0.668,0.689
MMS05_train_r2,0.270,0.708,0.754,0.705,0.525,0.430,0.423,0.416,0.422,0.437,0.412,0.431,0.701,0.462,0.524,0.460,0.513
MMS06_train_r2,0.423,0.634,0.814,0.665,0.750,0.513,0.477,0.460,0.474,0.595,0.512,0.593,0.630,0.622,0.652,0.634,0.670
MMS07_train_r2,0.320,0.360,0.544,0.399,0.891,0.797,0.743,nan,0.786,0.763,0.743,0.785,0.375,0.796,0.801,0.796,0.798
MMS09_train_r2,0.219,0.513,0.624,0.543,0.660,0.593,0.597,0.575,0.621,0.625,0.612,0.641,0.474,0.651,0.675,0.678,0.663
MMS11_train_r2,0.193,0.456,0.509,0.458,0.752,0.706,0.677,0.674,0.676,0.681,0.679,0.706,0.425,0.685,0.753,0.706,0.708
MMS12_train_r2,0.278,0.335,0.761,0.469,0.569,0.517,0.509,0.497,0.492,0.528,0.494,0.556,0.458,0.532,0.529,0.531,0.529
MMS13_train_r2,0.449,0.809,0.827,0.824,0.506,0.384,0.390,0.370,0.382,0.412,0.389,0.384,0.834,0.419,0.429,0.471,0.429


,SVR,MLR,N,FV,"FV_X(1,.5)","FV_X(1,.1)","FV_X(1,.01)","FV_X(1,.001)","FV_X(.1,.1)","FV_C(.1,.1)","FV_C(.1,.01)","FV_X(.5,.1) +C(.5,.1)",FVD,FVD2,"FVD2_X(1,.1)","FVD2_C(1,.1)","FVD2_X(.5,.1) +C(.5,.1)"
MMS01_test_r2,-0.316,-0.261,-5.444,-0.120,-0.329,-0.292,-0.532,0.011,-0.374,-0.152,-7.399,0.023,-0.289,-0.493,-0.015,-0.208,-0.400
MMS02_test_r2,0.290,0.398,0.130,0.187,0.166,0.262,0.477,0.265,0.230,0.114,0.390,0.416,0.320,0.366,0.415,0.492,0.433
MMS03_test_r2,-0.297,-0.261,-0.024,0.162,-0.433,-0.420,-0.598,-0.112,-0.103,0.019,-0.091,-0.404,0.025,-0.421,-0.576,-0.110,-0.331
MMS05_test_r2,-0.359,-0.062,-0.265,-0.193,-0.112,-0.225,-0.320,-0.258,-0.465,-0.128,-0.204,-0.044,-0.114,-0.004,-0.185,-0.533,-0.277
MMS06_test_r2,-0.055,-0.222,-7.382,-0.135,0.478,-0.324,0.320,0.281,0.027,-0.348,0.570,0.286,0.379,0.658,0.445,0.438,0.519
MMS07_test_r2,0.312,0.236,0.401,0.309,0.112,-0.200,0.448,nan,0.053,0.195,0.169,0.160,0.264,0.271,0.345,0.270,0.236
MMS09_test_r2,0.113,0.202,0.156,0.128,0.237,0.097,0.067,-0.299,0.150,0.176,0.156,0.165,0.179,0.194,0.231,0.236,0.219
MMS11_test_r2,-0.144,-1.222,-206.270,-1.311,-1.233,-0.969,-1.703,-1.169,-1.451,-1.621,-1.475,-1.055,-0.781,-0.819,-0.897,-1.145,-1.177
MMS12_test_r2,-0.582,-0.706,-2.089,-0.337,-0.642,-0.572,-0.503,-0.479,-0.433,-0.711,-0.394,-0.718,-2.593,-1.045,-0.621,-0.682,-0.662
MMS13_test_r2,0.193,0.471,0.647,0.513,0.649,0.526,0.537,0.532,0.528,0.403,0.602,0.546,0.560,0.524,0.576,0.592,0.553


# Expolation

In [2]:
import pandas as pd


res_dict = {
    # "SVR": "../scripts/res_svr.txt",
    # "MLR": "../scripts/res_mlr.txt",
    # "N": "../scripts/RUN_230306_N/res.txt",
    "FV": "../scripts/RUN_230306_EX_FV/res.txt",
    "FV_X(1,.5)": "../scripts/RUN_230306_EX_FV_X_5/res.txt",    
    "FV_X(.5,.1) +C(.5,.1)": "../scripts/RUN_230306_EX_FV_X5_1_C5_1/res.txt",
    "FVD2": "../scripts/RUN_230306_EX_FVD2/res.txt",
    # "FV_X": "../scripts/RUN_230306_FV/res.txt",
    # "FV_X(1,.1)": "../scripts/RUN_230224_FV_X_1/res.txt",
    # "FV_X(1,.01)": "../scripts/RUN_230224_FV_X_01/res.txt",
    # "FV_X(1,.001)": "../scripts/RUN_230224_FV_X_001/res.txt",
    # "FV_X(.1,.1)": "../scripts/RUN_230224_FV_X1_1/res.txt",
    # "FV_C(1,.1)": "../scripts/RUN_230224_FV_C_1/res.txt",
    # "FV_C(1,.01)": "../scripts/RUN_230224_FV_C_01/res.txt",
    # "FV_C(.1,.1)": "../scripts/RUN_230224_FV_C1_1/res.txt",
    # "FV_C(.1,.01)": "../scripts/RUN_230224_FV_C1_01/res.txt",
    # "FVD": "../scripts/RUN_230123_FVD/res.txt",
    # "FVD2": "../scripts/RUN_230224_FVD2/res.txt",
    # "FVD2_X(1,.1)": "../scripts/RUN_230224_FVD2_X_1/res.txt",
    # "FVD2_C(1,.1)": "../scripts/RUN_230224_FVD2_C_1/res.txt",
    # "FVD2_X(.5,.1) +C(.5,.1)": "../scripts/RUN_230224_FVD2_X5_1_C5_1/res.txt",
    # ###
}

df = pd.DataFrame()

for method, resfile in res_dict.items():

    print(method, resfile)
    with open(resfile) as f:
        ls = f.readlines()
        # print(ls)
        for line in ls:
            ds = line.split("\t")

            if "train_all" in ds[0]:
                continue

            # XXXXXX
            _val = ds[1].replace("inf", "10.0", 10)
            vals = eval(_val)
            # vals = eval(ds[1])

            row_rmse = "MMS"+ds[0]+"_rmse"
            row_r2 = "MMS"+ds[0]+"_r2"

            df.loc[row_rmse, method] = vals[0]
            df.loc[row_r2, method] = vals[1]


# df.index


FV ../scripts/RUN_230306_EX_FV/res.txt
FV_X(1,.5) ../scripts/RUN_230306_EX_FV_X_5/res.txt
FV_X(.5,.1) +C(.5,.1) ../scripts/RUN_230306_EX_FV_X5_1_C5_1/res.txt
FVD2 ../scripts/RUN_230306_EX_FVD2/res.txt


In [3]:
import matplotlib.pyplot as plt
import seaborn as sns

mmss = [f"{idx:02}" for idx in range(1, 16, 1)]
mtxs = ["rmse", "r2"]

"""remove duplication: MMS04, MMS08, MMS10 (MMS14 redo)"""
df = df.filter(regex="^(?!.*(MMS04|MMS08|MMS10)).*$", axis="index")
print(df.index)

DISPLAY_CHART=False
DO_SCATTER=False

if DISPLAY_CHART:
    for mms in mmss:

        print(mms)
        fig, axes = plt.subplots(1, 2, figsize=(16, 6))

        for idx, mtx in enumerate(mtxs):
            row_train = f"MMS{mms}_train_{mtx}"
            row_test = f"MMS{mms}_test_{mtx}"

            if row_train not in df.index or row_test not in df.index:
                continue

            _df = df.loc[[row_train,row_test],:].T

            if DO_SCATTER:
                # x1 = df[f"{mms}_train_{mtx}"]
                # x2 = df[f"{mms}_test_{mtx}"]

                _df.plot.scatter(x=row_train, y=row_test, ax=axes[idx], c=[_j for _j in range(len(list(_df.index)))], alpha=0.8, s=100)

                for _jdx, _label in enumerate(list(_df.index)):
                    axes[idx].text(_df.iloc[_jdx,0], _df.iloc[_jdx,1], _label, alpha=0.75)
                
                _xlim = axes[idx].get_xlim()
                _ylim = axes[idx].get_ylim()
                axes[idx].set_xlim(min(_xlim[0],_ylim[0]), max(_xlim[1],_ylim[1]))
                axes[idx].set_ylim(min(_xlim[0],_ylim[0]), max(_xlim[1],_ylim[1]))
                _xlim = axes[idx].get_xlim()
                _ylim = axes[idx].get_ylim()
                axes[idx].plot(_xlim, _ylim, color="black", lw=.5)
            else:
                _df.T.plot.bar(ax=axes[idx])
                # axes[idx].bar([x1, x2], label=[f"MMS{mms}_FV_{mtx}_train", f"MMS{mms}_FV_{mtx}_test"])


        plt.show()


"""setup result table"""
# display(df)

# pd.options.display.float_format = '{:.1f}'.format

df_train_rmse = df.filter(regex="train_rmse$", axis="index")
df_train_r2 = df.filter(regex="train_r2$", axis="index")
df_test_rmse = df.filter(regex="test_rmse$", axis="index")
df_test_r2 = df.filter(regex="test_r2$", axis="index")

# print(df_train_rmse.dtypes)
# _df = df_train_rmse.astype("float64")
# print(_df.dtypes)

cm_g  = sns.light_palette('green', reverse=True, as_cmap=True)
cm_gr = sns.light_palette('blue', reverse=False, as_cmap=True)
s_train_rmse = df_train_rmse.style.background_gradient(cmap=cm_g, axis=1).set_precision(3)
s_test_rmse = df_test_rmse.style.background_gradient(cmap=cm_g, axis=1).set_precision(3)
s_train_r2 = df_train_r2.style.background_gradient(cmap=cm_gr, vmax=1.0, axis=1).set_precision(3)
s_test_r2 = df_test_r2.style.background_gradient(cmap=cm_gr, vmax=1.0, axis=1).set_precision(3)

def max_bold_min_italic(val):
    is_max = val == val.max()
    is_min = val == val.min()
    ret = ['font-weight: bold' if cell else '' for cell in is_max]
    ret = ['font-style: italic' if cell else ret[idx] for idx, cell in enumerate(is_min)]
    return ret

def min_bold_max_italic(val):
    is_max = val == val.max()
    is_min= val == val.min()
    ret = ['font-weight: bold' if cell else '' for cell in is_min]
    ret = ['font-style: italic' if cell else ret[idx] for idx, cell in enumerate(is_max)]
    return ret

# pd.set_option("display.precision", 4)
# pd.options.display.precision = 6
# print(df_train_rmse.dtypes)


s_train_rmse = s_train_rmse.apply(min_bold_max_italic, axis=1)
s_test_rmse = s_test_rmse.apply(min_bold_max_italic, axis=1)
s_train_r2 = s_train_r2.apply(max_bold_min_italic, axis=1)
s_test_r2 = s_test_r2.apply(max_bold_min_italic, axis=1)

display(s_train_rmse)
display(s_test_rmse)
display(s_train_r2)
display(s_test_r2)
# df_train_rmse

Index(['MMS01_train_rmse', 'MMS01_train_r2', 'MMS01_test_rmse',
       'MMS01_test_r2'],
      dtype='object')


/tmp/ipykernel_3481/423571345.py:69: FutureWarning: this method is deprecated in favour of `Styler.format(precision=..)`
  s_train_rmse = df_train_rmse.style.background_gradient(cmap=cm_g, axis=1).set_precision(3)
/tmp/ipykernel_3481/423571345.py:70: FutureWarning: this method is deprecated in favour of `Styler.format(precision=..)`
  s_test_rmse = df_test_rmse.style.background_gradient(cmap=cm_g, axis=1).set_precision(3)
/tmp/ipykernel_3481/423571345.py:71: FutureWarning: this method is deprecated in favour of `Styler.format(precision=..)`
  s_train_r2 = df_train_r2.style.background_gradient(cmap=cm_gr, vmax=1.0, axis=1).set_precision(3)
/tmp/ipykernel_3481/423571345.py:72: FutureWarning: this method is deprecated in favour of `Styler.format(precision=..)`
  s_test_r2 = df_test_r2.style.background_gradient(cmap=cm_gr, vmax=1.0, axis=1).set_precision(3)


,FV,"FV_X(1,.5)","FV_X(.5,.1) +C(.5,.1)",FVD2
MMS01_train_rmse,0.537,0.537,0.537,0.608


,FV,"FV_X(1,.5)","FV_X(.5,.1) +C(.5,.1)",FVD2
MMS01_test_rmse,2.222,2.222,2.222,3.251


,FV,"FV_X(1,.5)","FV_X(.5,.1) +C(.5,.1)",FVD2
MMS01_train_r2,0.537,0.537,0.537,0.608


,FV,"FV_X(1,.5)","FV_X(.5,.1) +C(.5,.1)",FVD2
MMS01_test_r2,-13.635,-13.635,-13.635,-28.495
